In [1]:
import numpy as np
import sympy
from sympy import *
from scipy.optimize import anderson, newton_krylov, broyden1, broyden2

# Прямоугольник

In [2]:
def make_angle_between(x11, y11, x12, y12, x21, y21, x22, y22, angle):
    # Если важен направленный угол, то нужно сделать еще одно (или не одно?) уравнение
    
    a_x = x12 - x11
    a_y = y12 - y11
    b_x = x22 - x21
    b_y = y22 - y21
    
    prod = a_x * a_y + b_x * b_y
    len_a = sympy.sqrt(a_x ** 2 + a_y ** 2)
    len_b = sympy.sqrt(b_x ** 2 + b_y ** 2)
    
    cos = prod / (len_a * len_b)
    eq = Eq(cos, sympy.cos(angle))
    
    return eq

def make_length(x11, y11, x12, y12, length):
    dx = x12 - x11
    dy = y12 - y11
    len_ = sympy.sqrt(dx ** 2 + dy ** 2)
    eq = Eq(len_, length)
    return eq

<img src="https://content.screencast.com/users/Feldlime/folders/Jing/media/81360534-14e7-41ca-88a0-15044d2cca9b/2018-10-12_0925.png" align="left" width=300/>
<img src="https://content.screencast.com/users/Feldlime/folders/Jing/media/27d5f5d7-762a-4a0e-a87e-86dc9db25afc/2018-10-12_0929.png" align="left" width=300/>
<img src="https://content.screencast.com/users/Feldlime/folders/Jing/media/cf27c968-caec-4be4-b9b1-c63ccee1e1b3/2018-10-12_0940.png" width=300/>


**Делаем прямоугольник:**
1. Создаем 4 произвольных отрезка.
2. Связываем попарно их концы.
3. Закрепляем один из углов.
4. Делаем 3 из 4-х углов прямыми.
5. Закрепляем длины двух смежных сторон.

*Прямоугольник готов! (левая картинка)*
Он жесткий, его форму нельзя изменить, но он может вращаться вокруг закрепленного угла.

Теперь попробуем сделать прямым оставшийся угол (правая картинка).
Компас правильно понимает, что проблема именно в углах и подсвечивает их, остальные ограничения в порядке.
**Как он это понимает?**

Смотрим дальше.
Да, мы переопределили фигуру, но на самом деле ей ничто не мешает вращаться.
Но она не вращается. Видимо потому, что число степеней свободы формально сейчас 0.

Что интересно, если убрать закрепление точки (угла), то прямоугольник будет свободно двигаться, а подсветка ограничений останется прежней (что правильно).

In [3]:
from scipy.optimize import anderson, newton_krylov, broyden1, broyden2

In [4]:
from scipy.optimize import anderson
from time import time

n = 10

init = [1] * n

def f(x):
    ff = [sum([k * x_ for x_ in x]) for k in range(len(x))]
    ff[0] = x[0] ** 2 + 2 * x[1] ** 2 - 4
    ff[1] = x[2] ** 2 + 3 * x[3] ** 3 - 10
    return ff
    
start_time = time()
res = newton_krylov(f, init)

print(f'Elapsed: {time() - start_time:.3}')
print(f'Max error: {max(np.abs(f(res)))}')

Elapsed: 0.278
Max error: 1.9516087057525056e-07


## Точки и ограничения

In [5]:
# create ends of edges
points = {}
for n in range(1, 5):  # 4 edges
    for i in range(1, 3):  # 2 ends
        for coo in ['x', 'y']:
            name = f'{coo}{n}{i}'
            points[name] = symbols(name)
points

{'x11': x11,
 'y11': y11,
 'x12': x12,
 'y12': y12,
 'x21': x21,
 'y21': y21,
 'x22': x22,
 'y22': y22,
 'x31': x31,
 'y31': y31,
 'x32': x32,
 'y32': y32,
 'x41': x41,
 'y41': y41,
 'x42': x42,
 'y42': y42}

In [6]:
restrictions = {}

# combine points
for i in range(1, 5):
    for coo in 'xy':
        j = (i + 1) if i < 4 else 1
        restrictions[f'combine_{i}_{j}_{coo}'] = Eq(points[f'{coo}{i}2'], points[f'{coo}{j}1'])
        
# fix point in (0, 0)
for coo in 'xy':
    restrictions[f'fix_{coo}11'] = Eq(points[f'{coo}11'], 0)

# fix 3 right angles
for i, j in [(2, 3), (3, 4), (4, 1)]:
    points_ = [points[f'{coo}{k}{l}'] for k in [i, j] for l in [1, 2] for coo in 'xy']
    restrictions[f'right_angle_{i}_{j}'] = make_angle_between(*points_, sympy.pi / 2)
    
# fix 2 lengths (for 4mm)
for i in [1, 4]:
    points_ = [points[f'{coo}{i}{l}'] for l in [1, 2] for coo in 'xy']
    restrictions[f'length_{i}'] = make_length(*points_, 4)
    
restrictions

{'combine_1_2_x': Eq(x12, x21),
 'combine_1_2_y': Eq(y12, y21),
 'combine_2_3_x': Eq(x22, x31),
 'combine_2_3_y': Eq(y22, y31),
 'combine_3_4_x': Eq(x32, x41),
 'combine_3_4_y': Eq(y32, y41),
 'combine_4_1_x': Eq(x42, x11),
 'combine_4_1_y': Eq(y42, y11),
 'fix_x11': Eq(x11, 0),
 'fix_y11': Eq(y11, 0),
 'right_angle_2_3': Eq(((-x21 + x22)*(-y21 + y22) + (-x31 + x32)*(-y31 + y32))/(sqrt((-x21 + x22)**2 + (-y21 + y22)**2)*sqrt((-x31 + x32)**2 + (-y31 + y32)**2)), 0),
 'right_angle_3_4': Eq(((-x31 + x32)*(-y31 + y32) + (-x41 + x42)*(-y41 + y42))/(sqrt((-x31 + x32)**2 + (-y31 + y32)**2)*sqrt((-x41 + x42)**2 + (-y41 + y42)**2)), 0),
 'right_angle_4_1': Eq(((-x11 + x12)*(-y11 + y12) + (-x41 + x42)*(-y41 + y42))/(sqrt((-x11 + x12)**2 + (-y11 + y12)**2)*sqrt((-x41 + x42)**2 + (-y41 + y42)**2)), 0),
 'length_1': Eq(sqrt((-x11 + x12)**2 + (-y11 + y12)**2), 4),
 'length_4': Eq(sqrt((-x41 + x42)**2 + (-y41 + y42)**2), 4)}

## Попробуем решить систему уравнений из ограничений

In [7]:
system = [eq.lhs - eq.rhs for eq in restrictions.values()] 
symbols_ = list(points.values())

In [19]:
system.append(points['x12'] - 1)

In [24]:
system

[x12 - x21,
 y12 - y21,
 x22 - x31,
 y22 - y31,
 x32 - x41,
 y32 - y41,
 -x11 + x42,
 -y11 + y42,
 x11,
 y11,
 ((-x21 + x22)*(-y21 + y22) + (-x31 + x32)*(-y31 + y32))/(sqrt((-x21 + x22)**2 + (-y21 + y22)**2)*sqrt((-x31 + x32)**2 + (-y31 + y32)**2)),
 ((-x31 + x32)*(-y31 + y32) + (-x41 + x42)*(-y41 + y42))/(sqrt((-x31 + x32)**2 + (-y31 + y32)**2)*sqrt((-x41 + x42)**2 + (-y41 + y42)**2)),
 ((-x11 + x12)*(-y11 + y12) + (-x41 + x42)*(-y41 + y42))/(sqrt((-x11 + x12)**2 + (-y11 + y12)**2)*sqrt((-x41 + x42)**2 + (-y41 + y42)**2)),
 sqrt((-x11 + x12)**2 + (-y11 + y12)**2) - 4,
 sqrt((-x41 + x42)**2 + (-y41 + y42)**2) - 4,
 x12 - 1]

In [25]:
%%time
res = nonlinsolve(system, symbols_)

Wall time: 30.9 s


In [30]:
len(res)

12

In [31]:
for i, solution in enumerate(res):
    print('Solution', i)
    for sym, r in zip(symbols_, solution):
        print(sym, r.evalf())

Solution 0
x11 0
y11 0
x12 1.00000000000000
y12 -3.87298334620742
x21 1.00000000000000
y21 -3.87298334620742
x22 2.00000000000000
y22 0
x31 2.00000000000000
y31 0
x32 1.00000000000000
y32 3.87298334620742
x41 1.00000000000000
y41 3.87298334620742
x42 0
y42 0
Solution 1
x11 0
y11 0
x12 1.00000000000000
y12 3.87298334620742
x21 1.00000000000000
y21 3.87298334620742
x22 2.00000000000000
y22 0
x31 2.00000000000000
y31 0
x32 1.00000000000000
y32 -3.87298334620742
x41 1.00000000000000
y41 -3.87298334620742
x42 0
y42 0
Solution 2
x11 0
y11 0
x12 1.00000000000000
y12 3.87298334620742
x21 1.00000000000000
y21 3.87298334620742
x22 0
y22 7.74596669241483
x31 0
y31 7.74596669241483
x32 -1.00000000000000
y32 3.87298334620742
x41 -1.00000000000000
y41 3.87298334620742
x42 0
y42 0
Solution 3
x11 0
y11 0
x12 1.00000000000000
y12 -3.87298334620742
x21 1.00000000000000
y21 -3.87298334620742
x22 0
y22 -7.74596669241483
x31 0
y31 -7.74596669241483
x32 -1.00000000000000
y32 -3.87298334620742
x41 -1.0000000

In [21]:
def system_to_function(system, symbols):
    functions = [lambdify(symbols, f) for f in system]
    
    def fun(x):
        if len(x) != len(symbols):
            raise ValueError
        res = np.array([f(*x) for f in functions])
        return res
    
    return fun

In [43]:
system.pop(-1)

x12 - 1

In [46]:
system.append(system[0])

In [50]:
%%time
fun = system_to_function(system, symbols_)

init = np.random.random(len(symbols_))

res = newton_krylov(fun, init, verbose=False)
res

ValueError: Jacobian inversion yielded zero vector. This indicates a bug in the Jacobian approximation.

In [42]:
res


array([-1.25519536e-06,  1.12315845e-07,  1.00000096e+00,  3.87298173e+00,
        1.00000159e+00,  3.87298193e+00,  4.87298675e+00,  2.87297014e+00,
        4.87298767e+00,  2.87297013e+00,  3.87298291e+00, -1.00000426e+00,
        3.87298222e+00, -1.00000319e+00, -1.00494819e-06, -5.60668253e-07])